In [85]:
import triton
import triton.language as tl
import torch

In [86]:
@triton.jit
def kernel(
        input_ptr, output_ptr, buffer_ptr,
        N, seed, offset,
        BLOCK_SIZE: tl.constexpr
):
    pid = tl.program_id(0)
    offs = tl.arange(0, BLOCK_SIZE)
    input_offs = pid * BLOCK_SIZE + offs
    output_offs = pid * BLOCK_SIZE + offs
    buffer_offs = pid * BLOCK_SIZE + offs
    mask = input_offs<N
    inputdata = tl.load(input_ptr+input_offs, mask=mask)
    
    
    # tmp3 = tl.zeros((BLOCK_SIZE,), dtype=tl.float32)+2
#     seeds = tl.full([1, BLOCK_SIZE], seed, dtype=tl.int32)
#     tmp3 = tl.rand(seed, seed)
    mask = output_offs<N
    output_ptrs = output_ptr + output_offs

    tl.store(output_ptrs, inputdata, mask=mask)


In [87]:
def use_kernel(inputdata, buffer, seed, offset):
    output = torch.empty_like(inputdata)
    assert inputdata.is_cuda and buffer.is_cuda and output.is_cuda
    N = output.numel()
    grid = lambda META: (
        triton.cdiv(N, META['BLOCK_SIZE']),
    )
    kernel[grid](inputdata, output, buffer, N, seed, offset, BLOCK_SIZE=3)
    print(output)
    return output


In [88]:
N = 9
inputdata = torch.empty(N, device='cuda', dtype=torch.float32)
# output = torch.empty(N, device='cuda', dtype=torch.float32)
buffer = torch.empty(N, device='cuda', dtype=torch.float32)
seed=0
offset = 0
output = use_kernel(inputdata, buffer, seed, offset)

loc("/tmp/ipykernel_7604/3976588479.py":8:24): error: Number of elements must be power-of-two, but %2 = "tt.make_range"() <{end = 3 : i32, start = 0 : i32}> : () -> tensor<3xi32> doesn't follow the rule (3) elements


RuntimeError: PassManager::run failed